In [1]:
import json
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf

## Load Data

In [2]:
# Set Seaborn theme
sns.set_style('whitegrid')

# Location for the local data
path = './transit_vis/data/delay_modeling_datasets'

# Read urban variable shapefiles
street_data = pd.read_csv(f"{path}/Seattle_Streets.csv", low_memory=False)
bike_data = pd.read_csv(f"{path}/Existing_Bike_Facilities.csv", low_memory=False)
flow_data = pd.read_csv(f"{path}/2018_Traffic_Flow_Counts.csv", low_memory=False)
blockface_data = pd.read_csv(f"{path}/Blockface.csv", low_memory=False)

# Read data from summarize_rds
bus_data_all = pd.read_csv(f"{path}/../all_data.csv", chunksize=1000000, low_memory=False)
bus_data_df = pd.concat(bus_data_all)
del bus_data_all

# Filter datasets to variables of interest
street_data = street_data[['COMPKEY','ARTDESCRIPT','SPEEDLIMIT','ONEWAY','SEGLENGTH','SURFACEWIDTH','STREETTYPE',
                           'TRANDESCRIPT','SLOPE_PCT']].copy()
bike_data = bike_data[['COMPKEY','EXISTING_FACILITY_TYPE']].copy()
flow_data = flow_data[['COMPKEY','DOWNTOWN','AWDT']].copy()
blockface_data = blockface_data[['SEGKEY','LOAD','ZONE']].copy().groupby('SEGKEY').sum().reset_index()

# AWDT data has multiple compkeys per line comma separated
flow_data = flow_data.dropna()
ck = []
downtown = []
awdt = []
for i in range(0,len(flow_data)):
    keys = flow_data.iloc[i,0].split(',')
    for key in keys:
        ck.append(int(key))
        downtown.append(flow_data.iloc[i,1])
        awdt.append(flow_data.iloc[i,2])
flow_data = pd.DataFrame([ck,downtown,awdt])
flow_data = flow_data.transpose()
flow_data.columns = ['COMPKEY','DOWNTOWN','AWDT']

# AWDT also is recorded as strings
flow_data['AWDT'] = pd.to_numeric(flow_data['AWDT'], errors='coerce')
flow_data['COMPKEY'] = pd.to_numeric(flow_data['AWDT'], errors='coerce')
flow_data['COMPKEY'] = flow_data['COMPKEY'].astype('int')
flow_data['AWDT'] = flow_data['AWDT'].astype('int')

In [3]:
# Bus data needs to be aggregated by compkey and median/95th pct calculated for each segment
def percentile(n):
    def percentile_(x):
        return np.percentile(x, n)
    percentile_.__name__ = 'pct_%s' % n
    return percentile_

bus_data = bus_data_df.groupby(['seg_compkey']).agg(['median', 'var', 'count', percentile(95)]).reset_index()
bus_data = bus_data.loc[bus_data[('seg_route_id', 'count')] > 1]
x = bus_data[('seg_compkey')].values
y = bus_data[('avg_speed_m_s')][['median','pct_95']].iloc[:,0].values
z = bus_data[('avg_speed_m_s')][['median','pct_95']].iloc[:,1].values
bus_data = pd.DataFrame([x,y,z]).transpose()
bus_data.columns = ['seg_compkey','median','pct_95']
bus_data['performance'] = bus_data['median'] / bus_data['pct_95']

In [4]:
# Merge all datasets on Compkey
model_data = pd.merge(street_data, bike_data, how='left', on='COMPKEY')
model_data = pd.merge(model_data, flow_data, how='left', on='COMPKEY')
model_data = pd.merge(model_data, blockface_data, how='left', left_on='COMPKEY', right_on='SEGKEY')
model_data = pd.merge(model_data, bus_data, how='left', sort=True, left_on='COMPKEY', right_on='seg_compkey')

# Add case where there is no bike facility
model_data.loc[pd.isna(model_data['EXISTING_FACILITY_TYPE']), 'EXISTING_FACILITY_TYPE'] = 'No_Facility'

In [5]:
# There are still many NA values where records are not found in flow, blockface datasets, but do exist in streets
print(len(model_data))
model_data = model_data.dropna()
print(len(model_data))

27890
795


## Linear Regression

In [6]:
# All variables
results = smf.ols('performance ~ \
                  ARTDESCRIPT + \
                  SPEEDLIMIT + \
                  ONEWAY + \
                  SEGLENGTH + \
                  SURFACEWIDTH + \
                  STREETTYPE + \
                  TRANDESCRIPT + \
                  SLOPE_PCT + \
                  EXISTING_FACILITY_TYPE + \
                  DOWNTOWN + \
                  AWDT + \
                  LOAD + \
                  ZONE', data=model_data).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:            performance   R-squared:                       0.344
Model:                            OLS   Adj. R-squared:                  0.322
Method:                 Least Squares   F-statistic:                     15.50
Date:                Wed, 17 Feb 2021   Prob (F-statistic):           3.45e-54
Time:                        19:31:45   Log-Likelihood:                 552.19
No. Observations:                 795   AIC:                            -1050.
Df Residuals:                     768   BIC:                            -924.1
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
                                                            coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------